# Random numbers generation

## Linear congruential generators

Adapted from https://rosettacode.org/wiki/Linear_congruential_generator

In [ ]:
function getlcg(seed::Integer, a::Integer, c::Integer, m::Integer)
    state = seed
    am_mil = 1.0/m
    return function lcgrand()
        state = mod(a * state + c, m)
        return state*am_mil  # produce a number in (0,1)
    end
end

Standard minimal generator:

In [ ]:
stdmin = getlcg(1234, 16807, 0, 2^31-1)

In [ ]:
n = 10000

sample = zeros(n)

for i = 1:n
    sample[i] = stdmin()
end

In [ ]:
using Plots

In [ ]:
scatter(sample[[2*i+1 for i = 0:(Int)(n/2)-1]], sample[[2*i for i = 1:(Int)(n/2)]], label="", fmt = :png)

## RDST

Implementation of random streams.

In [ ]:
import Pkg

Pkg.add(url = "https://github.com/JLChartrand/RDST.jl")

In [ ]:
using RDST

### MRG32K3a

In [ ]:
mrg_gen1 = MRG32k3aGen([1,2,3,4,5,6])

In [ ]:
typeof(mrg_gen1)

In [ ]:
mrg_gen1 = MRG32k3aGen()

The `show` function prints the current seed of the generator.

In [ ]:
show(mrg_gen1)

In [ ]:
mrg_1 = next_stream(mrg_gen1)

In [ ]:
typeof(mrg_1)

In [ ]:
mrg_1 = next_stream(mrg_gen1)

In [ ]:
stream1a = [rand(mrg_1) for i in 1:10]
reset_substream!(mrg_1)
stream1b = [rand(mrg_1) for i in 1:10]
stream1a == stream1b

In [ ]:
show(mrg_gen1)

In [ ]:
next_substream!(mrg_1)

In [ ]:
stream2a = [rand(mrg_1) for i in 1:10]
reset_stream!(mrg_1)

In [ ]:
next_substream!(mrg_1)
stream2b = [rand(mrg_1) for i in 1:10]
stream2a == stream2b

In [ ]:
reset_stream!(mrg_1)
mrg_2 = next_stream(mrg_gen1)

## Nonuniform distributions

For continuous random variables, the inversion technique is equivalent to compute the quantile associated to the realization of a uniform random variable $U(0,1)$. We will use the distributions package.

In [ ]:
using Distributions

In [ ]:
N = Normal()
α = quantile(N, 0.975)

Normally distributed number generation:

In [ ]:
quantile(N, rand())

We can measure the required generation time with the package BenchmarkTools.

In [ ]:
using BenchmarkTools

In [ ]:
function InvertNormal()
    U = rand(Float64, 2)
    return quantile(N, U[1]), quantile(N, U[2])
end

In [ ]:
@btime X, Y = InvertNormal()

In [ ]:
function BoxMuller()
    U = rand(Float64, 2)
    
    R = sqrt(-2*log(U[1]))
    θ = 2*π*U[2]
    
    X = R*cos(θ)
    Y = R*sin(θ)
    
    return X, Y
end

In [ ]:
@btime X, Y = BoxMuller()

In [ ]:
randn()

In [ ]:
@btime randn()